In [1]:
import hoomd.htf as htf
import numpy as np
import tensorflow as tf
import hoomd
from hoomd import *
import os

In [2]:
graph = htf.graph_builder(0, output_forces=False)
# c = hoomd.context.initialize('--mode=gpu')
# system = hoomd.init.create_lattice(unitcell=hoomd.lattice.sq(a=1.0),
#                                     n=[3,3])
box = tf.Variable(tf.zeros_like(graph.box), validate_shape=False, name='box_size', trainable=False)
save_box_size = box.assign(graph.box)
graph.save_tensor(tf.identity(save_box_size),'BOX')
graph.save('skewed_box_size')






Note: Backed-up skewed_box_size previous model to skewed_box_size/previous_model_21



In [3]:
with htf.tfcompute('skewed_box_size') as tfcompute:
    hoomd.context.initialize("--mode=cpu")
    # create a square lattice
    rcut = 5
    system = hoomd.init.create_lattice(unitcell=hoomd.lattice.sq(a=4.0),
                                        n=[3,3])
    nlist = hoomd.md.nlist.cell()
    hoomd.md.integrate.mode_standard(dt=0.005)
    hoomd.md.integrate.nve(group=hoomd.group.all())
    tfcompute.attach(nlist, r_cut=rcut)
    hoomd.run(1e3)

notice(2): Started TF Session Manager.
HOOMD-blue v2.5.2 CUDA (10.0) DOUBLE HPMC_MIXED SSE SSE2 
Compiled: 04/23/2020
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 9 particles
notice(2): Force mode is FORCE_MODE.hoomd2tf 
notice(2): Starting TensorflowCompute 
notice(2): completed reallocate

INFO:tensorflow:The following quantities will computed:
INFO:tensorflow:	name period batch

INFO:tensorflow:	 AssignAdd

In [4]:
# Loading box dimensions
variables  = htf.load_variables('skewed_box_size', ['BOX'])
boxdim = variables['BOX']

print(boxdim[2])

INFO:tensorflow:Restoring parameters from skewed_box_size/model-999
[0. 0. 0.]


In [5]:
graph.wrap_vector(10,boxdim)
# tilt_value = [system.box.xy, system.box.xz, system.box.yz]

Checked: Box is not skewed in x axis.
Checked: Box is not skewed in y axis.
Checked: Box is not skewed in z axis.


<tf.Tensor 'sub_1:0' shape=(3,) dtype=float32>

In [ ]:

b=system.box
print (b.xy)
system.box.xz = 1
#print (system.box)
graph.wrap_vector(10)